<a href="https://colab.research.google.com/github/Ayush-07-cmd/Convolutional-Neural-Network-CNN-Network/blob/main/Convolutional_Neural_Network_CNN_Network(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Food-101 Image Classification in Google Colab — Full Explanation**

In [ ]:
#STEP 1: Install TensorFlow Datasets
#!pip install tensorflow-datasets

'''What this does:
We install tensorflow-datasets, a library that provides many pre-built datasets like Food-101, MNIST, CIFAR-10, etc., ready for use in ML models.'''

In [ ]:
#STEP 2: Import Required Libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np



# What this does:

tensorflow: Deep learning framework (we’ll use it to build/train our model)

tensorflow_datasets (tfds): To load the Food-101 dataset

matplotlib.pyplot: For image plotting

numpy: For numerical operations and predictions

In [ ]:
#STEP 3: Load the Food-101 Dataset
(ds_train, ds_test), ds_info = tfds.load(
    'food101',
    split=['train', 'validation'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)


# What this does step 3:

Loads the Food-101 dataset.

split=['train', 'validation']: Fetches training and test data.

as_supervised=True: Each sample is (image, label)

with_info=True: Returns metadata (like label names, shapes, etc.)

In [ ]:
#STEP 4: View Dataset Information
print("Total Classes:", ds_info.features['label'].num_classes)
print("Sample Classes:", ds_info.features['label'].names[:10])


# What this does step 4
ds_info: Metadata about the dataset

Prints number of food classes and some class names like "pizza", "sushi", etc.

In [ ]:
#STEP 5: Preprocess the Images
IMG_SIZE = 224
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))  # Resize to 224x224
    image = tf.cast(image, tf.float32) / 255.0             # Normalize pixels [0,1]
    return image, label

# Apply preprocessing to the datasets
ds_train = ds_train.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


# In step 5
Resizes images to 224x224 — standard input size for CNNs like MobileNet.

Normalizes pixel values (0–255 → 0–1).

Batching: Combines samples into groups of 32 for faster training.

Prefetching: Loads next batch while current is training (for speed).

In [ ]:
#STEP 6: Visualize Images
for images, labels in ds_train.take(1):
    plt.figure(figsize=(10, 10))
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy())
        label = ds_info.features['label'].int2str(labels[i].numpy())
        plt.title(label)
        plt.axis("off")
    plt.show()


# in step 6
Displays 9 food images from the training set.

Converts label integers to readable food names (e.g., "pizza").

In [ ]:
#STEP 7: Build the CNN Model (Transfer Learning)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False  # Freeze base layers

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),       # Flatten feature maps
    tf.keras.layers.Dense(101, activation='softmax')  # 101 food categories
])


# What In Step 7:

Loads MobileNetV2 pretrained on ImageNet (already knows general image features).

Freezes it to use as a feature extractor.

Adds a custom classification layer (Dense(101)) for our 101 food classes.

softmax: Returns probabilities for each class.

In [ ]:
#STEP 8: Compile the Model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


# What in step 8:
adam: Efficient optimizer for training.

sparse_categorical_crossentropy: Used for integer-labeled multi-class classification.

accuracy: Evaluation metric.

In [ ]:
#STEP 9: Train the Model
history = model.fit(ds_train, epochs=5, validation_data=ds_test)

# What in STEP 9:
Trains the model for 5 epochs (5 passes over training data).

Validates performance on the test set after each epoch.

In [ ]:
#STEP 10: Evaluate the Model
loss, accuracy = model.evaluate(ds_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


# What in step 10:
Tests final model on unseen data.

Prints accuracy — how well it classifies real food images.

In [ ]:
#STEP 11: Make Predictions & Show Results
class_names = ds_info.features['label'].names

for images, labels in ds_test.take(1):
    image = images[0]
    true_label = class_names[labels[0].numpy()]

    prediction = model.predict(tf.expand_dims(image, axis=0))
    predicted_label = class_names[np.argmax(prediction)]

    plt.imshow(image)
    plt.title(f"Predicted: {predicted_label}\nTrue: {true_label}")
    plt.axis('off')
    plt.show()


# What in step 11
Selects 1 image from test data.

Predicts its label using the model.

Displays the image with its true and predicted labels.

In [ ]:
#STEP 12: Save the Trained Model (Optional)
model.save("food101_model.h5")  # Save to file
#Saves the trained model in .h5 format so you can reuse it later.

Step	Code Block Purpose

1	Install Datasets

2	Import Libraries

3	Load Food-101

4	View Class Info

5	Preprocess Images

6	Visualize Samples

7	Build CNN Model

8	Compile Model

9	Train Model

10	Evaluate Model

11	Predict & Visualize

12	Save the Model

